In [1]:
# !pip install prometheus-api-client

In [26]:
import pandas as pd
from prometheus_api_client import PrometheusConnect

prometheus_url = "http://prometheus-server.monitoring:80"

prom = PrometheusConnect(url=prometheus_url, disable_ssl=True)

query = """
sum(
  (
	sum(
		sum(container_memory_allocation_bytes{job="kubecost"}) by (namespace,instance) * on(instance) group_left() (
				node_ram_hourly_cost{job="kubecost"} / 1024 / 1024 / 1024 * 730
				+ on(node,instance_type) group_left()
					label_replace
					(
						kube_node_labels{job="kubecost"}, "instance_type", "$1", "label_node_kubernetes_io_instance_type", "(.*)"
					) * 0
			)
  		+ 
  		sum(container_cpu_allocation{job="kubecost"}) by (namespace,instance) * on(instance) group_left() (
	  			node_cpu_hourly_cost{job="kubecost"} + on(node,instance_type) group_left()
		  			label_replace
		  			(
		  				kube_node_labels{job="kubecost"}, "instance_type", "$1", "label_node_kubernetes_io_instance_type", "(.*)"
		  			) * 0
		  	) * 730) by (namespace)
	) and on(namespace) kube_namespace_labels{label_billable="true", job="kubecost"}
) by (namespace)
"""

result = prom.custom_query(query)

df = pd.DataFrame(result)
df = pd.concat([df.drop(['metric'], axis=1), df['metric'].apply(pd.Series)], axis=1)

df['timestamp'] = pd.to_datetime(df['value'].apply(lambda x: x[0]), unit='s')

df['value'] = df['value'].apply(lambda x: float(x[1]))

df = df[['namespace', 'value', 'timestamp']]

df.head(5)

,namespace,value,timestamp
0,cgf,715.894266,2023-12-30 14:49:39.240
1,financas,179.624399,2023-12-30 14:49:39.240
2,ldi,76.197280,2023-12-30 14:49:39.240
3,snop,114.784202,2023-12-30 14:49:39.240
4,bcg,8.239581,2023-12-30 14:49:39.240
